# Data Analysis

In [ ]:
import cv2
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import json
import os
import sys
from collections import defaultdict

import random
import pandas as pd
import string


# Visualizzazione Dati

In [ ]:
df = pd.read_csv('VirtualData_CSV.csv.csv')


In [ ]:
print("Hard: ",len(df.loc[df['Difficulty'] == "hard"]))
print("Medium: ",len(df.loc[df['Difficulty'] == "medium"]))
print("Easy: ",len(df.loc[df['Difficulty'] == "easy"]))

In [ ]:
df["Model_name"].unique()

In [ ]:
dfv=df.loc[df['Difficulty'] == "hard"]
random_image=random.randint(a=0,b=len(dfv)-1)

In [ ]:
random_image

rgb_path=dfv.iloc[random_image]["RGB"]
right_line_path=dfv.iloc[random_image]["Good_Floor_Seg"]
wrong_line_path=dfv.iloc[random_image]["Wrong_Floor_Seg"]
rgb=cv2.imread(rgb_path)

fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 8))
right_line=cv2.imread(right_line_path)
cv2.imwrite("/home/rforte/NotebooksMisalignmentDetection/Saved/RightFloorSeg.jpg",right_line)

wrong_line=cv2.imread(wrong_line_path)
cv2.imwrite("/home/rforte/NotebooksMisalignmentDetection/Saved/WrongFloorSeg.jpg",wrong_line)

blended_right_line=cv2.addWeighted(rgb, 1, right_line, 1, 0)
blended_wrong_line=cv2.addWeighted(rgb, 1, wrong_line, 1, 0)
axs[0].imshow(blended_right_line[...,::-1])
axs[0].set_title("GOOD FLOOR")
axs[1].imshow(blended_wrong_line[...,::-1])
axs[1].set_title("WRONG FLOOR")

In [ ]:
def path_visualizer(difficulty):
    dfv=df.loc[df['Difficulty'] == difficulty]
    random_image=random.randint(a=0,b=len(dfv)-1)

    print("Record N°: ",random_image)
    print("Wrong pixel count: ",dfv.iloc[random_image]["Wrong_Floor_Pixel"])
    
    rgb_path=dfv.iloc[random_image]["RGB"]
    right_line_path=dfv.iloc[random_image]["Good_Line_Seg"]
    wrong_line_path=dfv.iloc[random_image]["Wrong_Line_Seg"]
    right_floor_path=dfv.iloc[random_image]["Good_Floor_Seg"]
    wrong_floor_path=dfv.iloc[random_image]["Wrong_Floor_Seg"]
    
    right_navmesh_path=dfv.iloc[random_image]["Good_Nav_Seg"]
    
    rgb=cv2.imread(rgb_path)
    
    
    right_line=cv2.imread(right_line_path)
    
    right_floor=cv2.imread(right_floor_path)
    right_navmesh=cv2.imread(right_navmesh_path)
    blended_right_line=cv2.addWeighted(rgb, 1, right_line, 1, 0)
    blended_right_floor=cv2.addWeighted(rgb, 1, right_floor, 1, 0)
    blended_right_floor_path=cv2.addWeighted(blended_right_floor,0.8, right_line, 1, 0)
    blended_right_navmesh=cv2.addWeighted(rgb, 1, right_navmesh, 1, 0)

    blended_right_nav_path=cv2.addWeighted(blended_right_navmesh,0.8, right_line, 1, 0)
    

    wrong_line=cv2.imread(wrong_line_path)
    wrong_floor=cv2.imread(wrong_floor_path)
    plt.imshow(wrong_floor[...,::-1])
    blended_wrong_line=cv2.addWeighted(rgb, 1, wrong_line, 1, 0)
    blended_wrong_floor=cv2.addWeighted(rgb, 1, wrong_floor, 1, 1)
    blended_wrong_floor_path=cv2.addWeighted(blended_right_floor, 1, wrong_line, 1, 0)
    
    fig, axs = plt.subplots(nrows=2, ncols=4, figsize=(10, 8))
    axs[0,0].imshow(rgb[...,::-1])
    axs[0,0].set_title("RGB IMAGE")
    axs[0,1].imshow(blended_right_line[...,::-1])
    axs[0,1].set_title("GOOD PATH")
    axs[0,2].imshow(blended_right_floor[...,::-1])
    axs[0,2].set_title("GOOD FLOOR")
    axs[0,3].imshow(blended_right_floor_path[...,::-1])
    axs[0,3].set_title("GOOD FLOOR+ GOOD PATH")
    
    axs[1,0].imshow(blended_right_nav_path[...,::-1])
    axs[1,0].set_title("GOOD NAV+ GOOD PATH")
    axs[1,1].imshow(blended_wrong_line[...,::-1])
    axs[1,1].set_title("WRONG PATH")
    axs[1,2].imshow(blended_wrong_floor[...,::-1])
    axs[1,2].set_title("WRONG FLOOR")
    axs[1,3].imshow(blended_wrong_floor_path[...,::-1])
    axs[1,3].set_title("GOOD FLOOR+ WRONG PATH")


    fig.suptitle(difficulty)
    np.set_printoptions(threshold=sys.maxsize)
    red=(right_line[:,:,0])

    print(len((red[...,::-1])[(red[...,::-1]>0)]))

In [ ]:
df

In [ ]:
random.seed()
path_visualizer("easy")

## creazione file txt di test e training

In [ ]:
def txt_creator(df,file_path):
    f=open(file_path, 'w')
    for row in df.iterrows():
        f.write(row[1]["RGB"]+", ")
        f.write(row[1]["Good_Line_Seg"]+", ")
        f.write(row[1]["Good_Floor_Seg"]+", ")
        f.write(row[1]["Wrong_Line_Seg"]+", ")
        f.write(row[1]["Wrong_Floor_Seg"]+"\n")

In [ ]:

from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.3)
val,test=train_test_split(test, test_size=0.5)
print(len(train), len(val), len(test))
txt_creator(train,"train.txt")
txt_creator(val,"validation.txt")
txt_creator(test,"test.txt")

In [ ]:
counts = df['Difficulty'].value_counts()
plt.bar(counts.index, counts.values)
plt.xlabel('Difficulty')
plt.ylabel('Elements')
plt.show()

In [ ]:
fig, axs = plt.subplots( ncols=2,nrows=1, figsize=(18, 8))

axs[0].hist(df['Good_Line_Pixel_On_Floor'],bins=50)
axs[0].set_title("Good Line Pixel On Floor")
axs[1].hist(df['Wrong_Line_Pixel_On_Floor'],bins=50)
axs[1].set_title("Wrong Line Pixel On Floor")

In [ ]:
N = 500
fig, axs = plt.subplots( ncols=2,nrows=1, figsize=(18, 8))

count, bins_count = np.histogram(df['Good_Line_Pixel_On_Floor'], bins=50)

pdf = count / sum(count)

cdf = np.cumsum(pdf)

axs[0].plot(bins_count[1:], pdf, color="red", label="PDF")
axs[0].plot(bins_count[1:], cdf, label="CDF")
axs[0].set_title("Good Line Pixel")
axs[0].grid()
count, bins_count = np.histogram(df['Wrong_Line_Pixel_On_Floor'], bins=50)

pdf = count / sum(count)

cdf = np.cumsum(pdf)

axs[1].plot(bins_count[1:], pdf, color="red", label="PDF")
axs[1].plot(bins_count[1:], cdf, label="CDF")
axs[1].set_title("Wrong Line Pixel")
axs[1].grid()

In [ ]:
fig, axs = plt.subplots( ncols=2,nrows=1, figsize=(10, 8))

axs[0].hist(df["Good_Line_Pixel_On_Floor"][df['Good_Line_Pixel_On_Floor']>=df['Wrong_Line_Pixel_On_Floor']],bins=50)
axs[0].set_title("Good Line >= Wrong Line "+str(len(df["Good_Line_Pixel_On_Floor"][df['Good_Line_Pixel_On_Floor']>=df['Wrong_Line_Pixel_On_Floor']])))
axs[1].hist(df["Good_Line_Pixel_On_Floor"][df['Good_Line_Pixel_On_Floor']<df['Wrong_Line_Pixel_On_Floor']],bins=50)
axs[1].set_title("Good Line < Wrong Line "+str(len(df["Good_Line_Pixel_On_Floor"][df['Good_Line_Pixel_On_Floor']<df['Wrong_Line_Pixel_On_Floor']])))